# Dataset Cleaning 

The main objectives of this notebook are: 
- Import dataset 
- Get familiar with the data 
- Determine if there are data quality issues 
- Resolve any data quality issues 

In [386]:
import polars as pl 
import polars.selectors as cs 
from google.cloud import bigquery
import numpy as np 

In [387]:
pl.Config.set_tbl_width_chars(200)

polars.config.Config

In [388]:
# Create client object
client = bigquery.Client()

In [389]:
QUERY = ("SELECT `date`, `visitStartTime`, `fullVisitorId`, `geoNetwork`.`subContinent`, `geoNetwork`.`country`,"
        "`geoNetwork`.`city`,`device`.`browser`, `device`.`operatingSystem`,`device`.`deviceCategory`,"
        "`trafficSource`.`source`, `h`.`item`.`transactionId`,`hp`.`v2ProductName`,`hp`.`v2ProductCategory` "
        "FROM `bigquery-public-data.google_analytics_sample.ga_sessions_20170731`, UNNEST(hits) AS h,"
        "UNNEST(h.product) AS hp;")

In [390]:
run_query = client.query(QUERY)

In [391]:
df = pl.from_arrow(run_query.to_arrow())

c:\Users\andre\AppData\Local\Programs\Python\Python312\Lib\site-packages\google\cloud\bigquery\table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


In [392]:
df.head()

date,visitStartTime,fullVisitorId,subContinent,country,city,browser,operatingSystem,deviceCategory,source,transactionId,v2ProductName,v2ProductCategory
str,i64,str,str,str,str,str,str,str,str,str,str,str
"""20170731""",1501520683,"""2219384770970157334""","""Eastern Europe""","""Slovakia""","""not available in demo dataset""","""Firefox""","""Windows""","""desktop""","""google""",null,"""Google Lunch Bag""","""Home/Bags/More Bags/"""
"""20170731""",1501520683,"""2219384770970157334""","""Eastern Europe""","""Slovakia""","""not available in demo dataset""","""Firefox""","""Windows""","""desktop""","""google""",null,"""Electronics Accessory Pouch""","""Home/Bags/More Bags/"""
"""20170731""",1501520683,"""2219384770970157334""","""Eastern Europe""","""Slovakia""","""not available in demo dataset""","""Firefox""","""Windows""","""desktop""","""google""",null,"""Sport Bag""","""Home/Bags/More Bags/"""
"""20170731""",1501520683,"""2219384770970157334""","""Eastern Europe""","""Slovakia""","""not available in demo dataset""","""Firefox""","""Windows""","""desktop""","""google""",null,"""Google Slim Utility Travel Bag""","""Home/Bags/More Bags/"""
"""20170731""",1501520683,"""2219384770970157334""","""Eastern Europe""","""Slovakia""","""not available in demo dataset""","""Firefox""","""Windows""","""desktop""","""google""",null,"""Suitcase Organizer Cubes""","""Home/Bags/More Bags/"""


In [393]:
df.shape

(49512, 13)

In [394]:
df.columns

['date',
 'visitStartTime',
 'fullVisitorId',
 'subContinent',
 'country',
 'city',
 'browser',
 'operatingSystem',
 'deviceCategory',
 'source',
 'transactionId',
 'v2ProductName',
 'v2ProductCategory']

In [395]:
df.sample()

date,visitStartTime,fullVisitorId,subContinent,country,city,browser,operatingSystem,deviceCategory,source,transactionId,v2ProductName,v2ProductCategory
str,i64,str,str,str,str,str,str,str,str,str,str,str
"""20170731""",1501487817,"""1904971737917285354""","""Northern Europe""","""United Kingdom""","""not available in demo dataset""","""Chrome""","""Chrome OS""","""desktop""","""google""",null,"""YouTube Custom Decals""","""Home/Shop by Brand/YouTube/"""


In [396]:
df.describe()

statistic,date,visitStartTime,fullVisitorId,subContinent,country,city,browser,operatingSystem,deviceCategory,source,transactionId,v2ProductName,v2ProductCategory
str,str,f64,str,str,str,str,str,str,str,str,str,str,str
"""count""","""49512""",49512.0,"""49512""","""49512""","""49512""","""49512""","""49512""","""49512""","""49512""","""49512""","""462""","""49512""","""49512"""
"""null_count""","""0""",0.0,"""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""49050""","""0""","""0"""
"""mean""",null,1.5015e9,null,null,null,null,null,null,null,null,null,null,null
"""std""",null,21502.167394,null,null,null,null,null,null,null,null,null,null,null
"""min""","""20170731""",1.5015e9,"""0002457163364254438""","""Australasia""","""Algeria""","""(not set)""","""Amazon Silk""","""(not set)""","""desktop""","""(direct)""","""ORD201707311786""","""1 oz Hand Sanitizer""","""${escCatTitle}"""
"""25%""",null,1.5015e9,null,null,null,null,null,null,null,null,null,null,null
"""50%""",null,1.5015e9,null,null,null,null,null,null,null,null,null,null,null
"""75%""",null,1.5015e9,null,null,null,null,null,null,null,null,null,null,null
"""max""","""20170731""",1.5016e9,"""9997362993085245352""","""Western Europe""","""Vietnam""","""not available in demo dataset""","""YaBrowser""","""iOS""","""tablet""","""youtube.com""","""ORD201707312663""","""YouTube Youth Short Sleeve Tee…","""Wearables/Men's T-Shirts/"""


In [397]:
df.select(pl.all().is_null().sum())

date,visitStartTime,fullVisitorId,subContinent,country,city,browser,operatingSystem,deviceCategory,source,transactionId,v2ProductName,v2ProductCategory
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,0,0,0,0,0,49050,0,0


In [398]:
df.filter(pl.col('transactionId')!='null')

date,visitStartTime,fullVisitorId,subContinent,country,city,browser,operatingSystem,deviceCategory,source,transactionId,v2ProductName,v2ProductCategory
str,i64,str,str,str,str,str,str,str,str,str,str,str
"""20170731""",1501525374,"""186231215995844689""","""Northern America""","""United States""","""Mountain View""","""Chrome""","""Macintosh""","""desktop""","""(direct)""","""ORD201707311786""","""Google Men's Bike Short Sleeve…","""Apparel"""
"""20170731""",1501525374,"""186231215995844689""","""Northern America""","""United States""","""Mountain View""","""Chrome""","""Macintosh""","""desktop""","""(direct)""","""ORD201707311786""","""Google Men's Bike Short Sleeve…","""Apparel"""
"""20170731""",1501549028,"""0345672699449577691""","""Northern America""","""United States""","""Kirkland""","""Chrome""","""Chrome OS""","""desktop""","""(direct)""","""ORD201707312107""","""Google Blackout Cap""","""Headgear"""
"""20170731""",1501549028,"""0345672699449577691""","""Northern America""","""United States""","""Kirkland""","""Chrome""","""Chrome OS""","""desktop""","""(direct)""","""ORD201707312107""","""Google Women's Vintage Hero Te…","""Apparel"""
"""20170731""",1501549028,"""0345672699449577691""","""Northern America""","""United States""","""Kirkland""","""Chrome""","""Chrome OS""","""desktop""","""(direct)""","""ORD201707312107""","""Google Men's Long & Lean Tee C…","""Apparel"""
…,…,…,…,…,…,…,…,…,…,…,…,…
"""20170731""",1501537217,"""7483600664917507409""","""Northern America""","""United States""","""Mountain View""","""Chrome""","""Macintosh""","""desktop""","""(direct)""","""ORD201707312663""","""Google 25 oz Red Stainless Ste…","""Drinkware"""
"""20170731""",1501537217,"""7483600664917507409""","""Northern America""","""United States""","""Mountain View""","""Chrome""","""Macintosh""","""desktop""","""(direct)""","""ORD201707312663""","""Android 17oz Stainless Steel S…","""Drinkware"""
"""20170731""",1501537217,"""7483600664917507409""","""Northern America""","""United States""","""Mountain View""","""Chrome""","""Macintosh""","""desktop""","""(direct)""","""ORD201707312663""","""Android Men's Short Sleeve Her…","""Apparel"""


In [399]:
df_cleaned = df.with_columns(pl.col('date').str.strptime(pl.Date, format="%Y%m%d"))
df_cleaned.head()

date,visitStartTime,fullVisitorId,subContinent,country,city,browser,operatingSystem,deviceCategory,source,transactionId,v2ProductName,v2ProductCategory
date,i64,str,str,str,str,str,str,str,str,str,str,str
2017-07-31,1501520683,"""2219384770970157334""","""Eastern Europe""","""Slovakia""","""not available in demo dataset""","""Firefox""","""Windows""","""desktop""","""google""",null,"""Google Lunch Bag""","""Home/Bags/More Bags/"""
2017-07-31,1501520683,"""2219384770970157334""","""Eastern Europe""","""Slovakia""","""not available in demo dataset""","""Firefox""","""Windows""","""desktop""","""google""",null,"""Electronics Accessory Pouch""","""Home/Bags/More Bags/"""
2017-07-31,1501520683,"""2219384770970157334""","""Eastern Europe""","""Slovakia""","""not available in demo dataset""","""Firefox""","""Windows""","""desktop""","""google""",null,"""Sport Bag""","""Home/Bags/More Bags/"""
2017-07-31,1501520683,"""2219384770970157334""","""Eastern Europe""","""Slovakia""","""not available in demo dataset""","""Firefox""","""Windows""","""desktop""","""google""",null,"""Google Slim Utility Travel Bag""","""Home/Bags/More Bags/"""
2017-07-31,1501520683,"""2219384770970157334""","""Eastern Europe""","""Slovakia""","""not available in demo dataset""","""Firefox""","""Windows""","""desktop""","""google""",null,"""Suitcase Organizer Cubes""","""Home/Bags/More Bags/"""


In [400]:
df_cleaned = df_cleaned.with_columns(pl.when(pl.col('transactionId') != 'null').then(1).otherwise(0).alias("transactionId"))
df_cleaned

date,visitStartTime,fullVisitorId,subContinent,country,city,browser,operatingSystem,deviceCategory,source,transactionId,v2ProductName,v2ProductCategory
date,i64,str,str,str,str,str,str,str,str,i32,str,str
2017-07-31,1501520683,"""2219384770970157334""","""Eastern Europe""","""Slovakia""","""not available in demo dataset""","""Firefox""","""Windows""","""desktop""","""google""",0,"""Google Lunch Bag""","""Home/Bags/More Bags/"""
2017-07-31,1501520683,"""2219384770970157334""","""Eastern Europe""","""Slovakia""","""not available in demo dataset""","""Firefox""","""Windows""","""desktop""","""google""",0,"""Electronics Accessory Pouch""","""Home/Bags/More Bags/"""
2017-07-31,1501520683,"""2219384770970157334""","""Eastern Europe""","""Slovakia""","""not available in demo dataset""","""Firefox""","""Windows""","""desktop""","""google""",0,"""Sport Bag""","""Home/Bags/More Bags/"""
2017-07-31,1501520683,"""2219384770970157334""","""Eastern Europe""","""Slovakia""","""not available in demo dataset""","""Firefox""","""Windows""","""desktop""","""google""",0,"""Google Slim Utility Travel Bag""","""Home/Bags/More Bags/"""
2017-07-31,1501520683,"""2219384770970157334""","""Eastern Europe""","""Slovakia""","""not available in demo dataset""","""Firefox""","""Windows""","""desktop""","""google""",0,"""Suitcase Organizer Cubes""","""Home/Bags/More Bags/"""
…,…,…,…,…,…,…,…,…,…,…,…,…
2017-07-31,1501537217,"""7483600664917507409""","""Northern America""","""United States""","""Mountain View""","""Chrome""","""Macintosh""","""desktop""","""(direct)""",1,"""Android 17oz Stainless Steel S…","""Drinkware"""
2017-07-31,1501537217,"""7483600664917507409""","""Northern America""","""United States""","""Mountain View""","""Chrome""","""Macintosh""","""desktop""","""(direct)""",1,"""Android Men's Short Sleeve Her…","""Apparel"""
2017-07-31,1501537217,"""7483600664917507409""","""Northern America""","""United States""","""Mountain View""","""Chrome""","""Macintosh""","""desktop""","""(direct)""",1,"""Android Men's Take Charge Shor…","""Apparel"""


In [401]:
encoding = df_cleaned.filter(pl.col('country') == "United States").select(
    pl.col('fullVisitorId'),pl.col('country'), pl.col('browser'),
    pl.col('transactionId'),pl.col('v2ProductName')
)
encoding

fullVisitorId,country,browser,transactionId,v2ProductName
str,str,str,i32,str
"""9308310352918219134""","""United States""","""Chrome""",0,"""Google Snapback Hat Black"""
"""9308310352918219134""","""United States""","""Chrome""",0,"""Android Wool Heather Cap Heath…"
"""9308310352918219134""","""United States""","""Chrome""",0,"""Google Blackout Cap"""
"""9308310352918219134""","""United States""","""Chrome""",0,"""Google 5-Panel Snapback Cap"""
"""9308310352918219134""","""United States""","""Chrome""",0,"""Android 5-Panel Low Cap"""
…,…,…,…,…
"""7483600664917507409""","""United States""","""Chrome""",1,"""Android 17oz Stainless Steel S…"
"""7483600664917507409""","""United States""","""Chrome""",1,"""Android Men's Short Sleeve Her…"
"""7483600664917507409""","""United States""","""Chrome""",1,"""Android Men's Take Charge Shor…"


In [402]:
# encoding = encoding.with_columns(pl.col('transactionId').mean().alias('transactionId'))

In [403]:
canada = df_cleaned.filter(pl.col('country') == "Canada").select(
    pl.col('fullVisitorId'),pl.col('country'), pl.col('browser'),
    pl.col('transactionId'),pl.col('v2ProductName')
)
# canada = canada.with_columns(pl.col('transactionId').mean().alias('transactionId'))
canada

fullVisitorId,country,browser,transactionId,v2ProductName
str,str,str,i32,str
"""6933896823600086395""","""Canada""","""Chrome""",0,"""Gift Card- $100.00"""
"""6933896823600086395""","""Canada""","""Chrome""",0,"""Gift Card - $25.00"""
"""6933896823600086395""","""Canada""","""Chrome""",0,"""Gift Card - $250.00"""
"""6933896823600086395""","""Canada""","""Chrome""",0,"""Gift Card - $50.00"""
"""7700062233308774111""","""Canada""","""Chrome""",0,"""Google Men's Long Sleeve Ragla…"
…,…,…,…,…
"""7444499703471460562""","""Canada""","""Chrome""",0,"""Google Women's Quilted Insulat…"
"""7444499703471460562""","""Canada""","""Chrome""",0,"""Google Women's Quilted Insulat…"
"""7444499703471460562""","""Canada""","""Chrome""",0,"""Google Women's 1/4 Zip Jacket …"


In [404]:
new_df = encoding.vstack(canada)
new_df

fullVisitorId,country,browser,transactionId,v2ProductName
str,str,str,i32,str
"""9308310352918219134""","""United States""","""Chrome""",0,"""Google Snapback Hat Black"""
"""9308310352918219134""","""United States""","""Chrome""",0,"""Android Wool Heather Cap Heath…"
"""9308310352918219134""","""United States""","""Chrome""",0,"""Google Blackout Cap"""
"""9308310352918219134""","""United States""","""Chrome""",0,"""Google 5-Panel Snapback Cap"""
"""9308310352918219134""","""United States""","""Chrome""",0,"""Android 5-Panel Low Cap"""
…,…,…,…,…
"""7444499703471460562""","""Canada""","""Chrome""",0,"""Google Women's Quilted Insulat…"
"""7444499703471460562""","""Canada""","""Chrome""",0,"""Google Women's Quilted Insulat…"
"""7444499703471460562""","""Canada""","""Chrome""",0,"""Google Women's 1/4 Zip Jacket …"


In [405]:
new_df = new_df.with_columns(pl.when(pl.col('country') != 'United States').then(1).otherwise(0).alias("country"))
new_df

fullVisitorId,country,browser,transactionId,v2ProductName
str,i32,str,i32,str
"""9308310352918219134""",0,"""Chrome""",0,"""Google Snapback Hat Black"""
"""9308310352918219134""",0,"""Chrome""",0,"""Android Wool Heather Cap Heath…"
"""9308310352918219134""",0,"""Chrome""",0,"""Google Blackout Cap"""
"""9308310352918219134""",0,"""Chrome""",0,"""Google 5-Panel Snapback Cap"""
"""9308310352918219134""",0,"""Chrome""",0,"""Android 5-Panel Low Cap"""
…,…,…,…,…
"""7444499703471460562""",1,"""Chrome""",0,"""Google Women's Quilted Insulat…"
"""7444499703471460562""",1,"""Chrome""",0,"""Google Women's Quilted Insulat…"
"""7444499703471460562""",1,"""Chrome""",0,"""Google Women's 1/4 Zip Jacket …"


In [406]:
new_df.select(pl.col('v2ProductName')).unique()

v2ProductName
str
"""Google Women's Shell Jacket Bl…"
"""Colored Pencil Set"""
"""Google 22 oz Water Bottle"""
"""Rocket Flashlight"""
"""Android Men's Take Charge Shor…"
…
"""Android Women's Long Sleeve Bl…"
"""22 oz YouTube Bottle Infuser"""
"""Gunmetal Roller Ball Pen"""


In [407]:
from sklearn.preprocessing import LabelEncoder

In [408]:
le = LabelEncoder()
le.fit(new_df.select(pl.col('v2ProductName')))

c:\Users\andre\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\preprocessing\_label.py:97: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LabelEncoder()

In [409]:
encoded_products = le.transform(new_df.select(pl.col('v2ProductName')))
encoded_products

c:\Users\andre\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\preprocessing\_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


array([122,  37,  66, ..., 136, 145, 166])

In [410]:
new_df= new_df.with_columns(pl.Series('v2ProductName', encoded_products))
new_df

fullVisitorId,country,browser,transactionId,v2ProductName
str,i32,str,i32,i32
"""9308310352918219134""",0,"""Chrome""",0,122
"""9308310352918219134""",0,"""Chrome""",0,37
"""9308310352918219134""",0,"""Chrome""",0,66
"""9308310352918219134""",0,"""Chrome""",0,62
"""9308310352918219134""",0,"""Chrome""",0,9
…,…,…,…,…
"""7444499703471460562""",1,"""Chrome""",0,146
"""7444499703471460562""",1,"""Chrome""",0,147
"""7444499703471460562""",1,"""Chrome""",0,136


In [411]:
new_df

fullVisitorId,country,browser,transactionId,v2ProductName
str,i32,str,i32,i32
"""9308310352918219134""",0,"""Chrome""",0,122
"""9308310352918219134""",0,"""Chrome""",0,37
"""9308310352918219134""",0,"""Chrome""",0,66
"""9308310352918219134""",0,"""Chrome""",0,62
"""9308310352918219134""",0,"""Chrome""",0,9
…,…,…,…,…
"""7444499703471460562""",1,"""Chrome""",0,146
"""7444499703471460562""",1,"""Chrome""",0,147
"""7444499703471460562""",1,"""Chrome""",0,136


In [412]:
le.fit(df.select(pl.col('browser')))

c:\Users\andre\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\preprocessing\_label.py:97: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LabelEncoder()

In [413]:
encoded_products = le.transform(new_df.select(pl.col('browser')))
encoded_products

c:\Users\andre\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\preprocessing\_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


array([2, 2, 2, ..., 2, 2, 2])

In [414]:
product_vector= new_df.with_columns(pl.Series('browser', encoded_products))
product_vector

fullVisitorId,country,browser,transactionId,v2ProductName
str,i32,i32,i32,i32
"""9308310352918219134""",0,2,0,122
"""9308310352918219134""",0,2,0,37
"""9308310352918219134""",0,2,0,66
"""9308310352918219134""",0,2,0,62
"""9308310352918219134""",0,2,0,9
…,…,…,…,…
"""7444499703471460562""",1,2,0,146
"""7444499703471460562""",1,2,0,147
"""7444499703471460562""",1,2,0,136


In [415]:
user_profiles = product_vector.group_by(pl.col('fullVisitorId')).agg(
    pl.col('country'), pl.col('browser'),
    pl.col('transactionId'), pl.col('v2ProductName')
)
user_profiles

fullVisitorId,country,browser,transactionId,v2ProductName
str,list[i32],list[i32],list[i32],list[i32]
"""5984333398882561849""","[0, 0, … 0]","[2, 2, … 2]","[0, 0, … 0]","[216, 2, … 215]"
"""6546241663577381502""","[0, 0, … 0]","[10, 10, … 10]","[0, 0, … 0]","[216, 2, … 14]"
"""7182381503550452083""","[0, 0, … 0]","[2, 2, … 2]","[0, 0, … 0]","[216, 2, … 132]"
"""4260799386429749980""","[0, 0, … 0]","[2, 2, … 2]","[0, 0, … 0]","[48, 74, … 185]"
"""9877271367690995601""","[0, 0, … 0]","[2, 2, … 2]","[0, 0, … 0]","[7, 28, … 210]"
…,…,…,…,…
"""7044385842215655813""","[0, 0, … 0]","[2, 2, … 2]","[0, 0, … 0]","[84, 120, … 143]"
"""9689211469745987567""","[0, 0, … 0]","[2, 2, … 2]","[0, 0, … 0]","[213, 41, … 28]"
"""9442478693509373201""","[0, 0, 0]","[2, 2, 2]","[0, 0, 0]","[47, 6, 69]"


In [416]:
from sklearn.metrics.pairwise import cosine_similarity

In [417]:
user_profiles= user_profiles.drop('fullVisitorId')
user_profiles

country,browser,transactionId,v2ProductName
list[i32],list[i32],list[i32],list[i32]
"[0, 0, … 0]","[2, 2, … 2]","[0, 0, … 0]","[216, 2, … 215]"
"[0, 0, … 0]","[10, 10, … 10]","[0, 0, … 0]","[216, 2, … 14]"
"[0, 0, … 0]","[2, 2, … 2]","[0, 0, … 0]","[216, 2, … 132]"
"[0, 0, … 0]","[2, 2, … 2]","[0, 0, … 0]","[48, 74, … 185]"
"[0, 0, … 0]","[2, 2, … 2]","[0, 0, … 0]","[7, 28, … 210]"
…,…,…,…
"[0, 0, … 0]","[2, 2, … 2]","[0, 0, … 0]","[84, 120, … 143]"
"[0, 0, … 0]","[2, 2, … 2]","[0, 0, … 0]","[213, 41, … 28]"
"[0, 0, 0]","[2, 2, 2]","[0, 0, 0]","[47, 6, 69]"


In [429]:
user_profiles = np.array(user_profiles)
user_profiles[0]

array([array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
       array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]),
       array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
       array([216,   2, 224, 217, 222, 223, 221, 220, 219, 226, 227, 215])],
      dtype=object)

In [419]:
product_vector = product_vector.drop('fullVisitorId')
# product_vector = np.array(product_vector)
product_vector

country,browser,transactionId,v2ProductName
i32,i32,i32,i32
0,2,0,122
0,2,0,37
0,2,0,66
0,2,0,62
0,2,0,9
…,…,…,…
1,2,0,146
1,2,0,147
1,2,0,136


In [420]:
test_vector = np.array(product_vector)
test_vector

array([[  0,   2,   0, 122],
       [  0,   2,   0,  37],
       [  0,   2,   0,  66],
       ...,
       [  1,   2,   0, 136],
       [  1,   2,   0, 145],
       [  1,   2,   0, 166]])

In [ ]:
break

In [431]:
user_product_similarity = cosine_similarity(user_profiles[0][0], test_vector)
user_product_similarity

ValueError: Expected 2D array, got 1D array instead:
array=[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [ ]:
new_df.write_csv("../data/cleaned_google_analytics.csv")

In [ ]:
# Need to split the data into subsets & perform the target encoding... k fold target encoding
# num_samples = len(new_df) // 8
# num_samples
# new_df = new_df.with_columns(pl.DataFrame({'kfold':np.repeat(np.arange(1, 9), num_samples)}))
# new_df

4087

### Data Pre-processing Pipeline

In [ ]:
def preprocessing_pipeline(df: pl.DataFrame) -> pl.DataFrame:
    """
    Args:
        df : pl.DataFrame

    Returns:
        pl.DataFrames: pre-processed dataframe
    """

    df = df.with_columns(pl.col('date').str.strptime(pl.Date, format="%Y%m%d"))
    df.write_csv("../data/cleaned_google_analytics.csv")
    return df 

In [ ]:
preprocessing_pipeline(df)

date,visitStartTime,fullVisitorId,subContinent,country,city,browser,operatingSystem,deviceCategory,source,transactionId,v2ProductName,v2ProductCategory
date,i64,str,str,str,str,str,str,str,str,str,str,str
2017-07-31,1501520683,"""2219384770970157334""","""Eastern Europe""","""Slovakia""","""not available in demo dataset""","""Firefox""","""Windows""","""desktop""","""google""",null,"""Google Lunch Bag""","""Home/Bags/More Bags/"""
2017-07-31,1501520683,"""2219384770970157334""","""Eastern Europe""","""Slovakia""","""not available in demo dataset""","""Firefox""","""Windows""","""desktop""","""google""",null,"""Electronics Accessory Pouch""","""Home/Bags/More Bags/"""
2017-07-31,1501520683,"""2219384770970157334""","""Eastern Europe""","""Slovakia""","""not available in demo dataset""","""Firefox""","""Windows""","""desktop""","""google""",null,"""Sport Bag""","""Home/Bags/More Bags/"""
2017-07-31,1501520683,"""2219384770970157334""","""Eastern Europe""","""Slovakia""","""not available in demo dataset""","""Firefox""","""Windows""","""desktop""","""google""",null,"""Google Slim Utility Travel Bag""","""Home/Bags/More Bags/"""
2017-07-31,1501520683,"""2219384770970157334""","""Eastern Europe""","""Slovakia""","""not available in demo dataset""","""Firefox""","""Windows""","""desktop""","""google""",null,"""Suitcase Organizer Cubes""","""Home/Bags/More Bags/"""
…,…,…,…,…,…,…,…,…,…,…,…,…
2017-07-31,1501537217,"""7483600664917507409""","""Northern America""","""United States""","""Mountain View""","""Chrome""","""Macintosh""","""desktop""","""(direct)""","""ORD201707312663""","""Android 17oz Stainless Steel S…","""Drinkware"""
2017-07-31,1501537217,"""7483600664917507409""","""Northern America""","""United States""","""Mountain View""","""Chrome""","""Macintosh""","""desktop""","""(direct)""","""ORD201707312663""","""Android Men's Short Sleeve Her…","""Apparel"""
2017-07-31,1501537217,"""7483600664917507409""","""Northern America""","""United States""","""Mountain View""","""Chrome""","""Macintosh""","""desktop""","""(direct)""","""ORD201707312663""","""Android Men's Take Charge Shor…","""Apparel"""


# Conclusions 
- 